In [0]:
from google.colab import auth
auth.authenticate_user()


from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Embedding
from keras.layers import LSTM,GRU
from keras.layers import Conv1D,MaxPooling1D,Conv2D,MaxPooling2D,GlobalMaxPooling2D,Bidirectional,Reshape,Dropout
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
np.random.seed(3)

In [0]:
np.random.seed(3)
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/valve_id_00_+6db/trainingSet',
                                                 target_size = (560, 420),
                                                 batch_size = 12,                                                
                                                 class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/valve_id_00_+6db/testSet',
                                            target_size = (560, 420),
                                            batch_size = 8,                                            
                                            class_mode = 'categorical')

Found 120 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [0]:
model = Sequential()
model.add(Conv2D(28, kernel_size=(2, 2),
                 activation='relu',input_shape=(560,420,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(15, kernel_size=(1,1),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(1, kernel_size=(1,1),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Reshape((-1,1)))
model.add(GRU(200, activation='relu',return_sequences=True))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax' ))
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001),metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 559, 419, 28)      364       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 279, 209, 28)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 279, 209, 28)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 279, 209, 15)      435       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 139, 104, 15)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 139, 104, 1)       16        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 69, 52, 1)        

In [0]:
early_stopping_callback = EarlyStopping(monitor='val_loss',mode='min', patience=5)
checkpoint_callback = ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=10,
        validation_data=test_generator,
        validation_steps=10,
        callbacks=[early_stopping_callback, checkpoint_callback])

Epoch 1/10
10/10 [==============================] - 64s 6s/step - loss: 0.6893 - acc: 0.5750 - val_loss: 0.6544 - val_acc: 0.6750

Epoch 00001: val_loss improved from inf to 0.65439, saving model to best_model.h5
Epoch 2/10
10/10 [==============================] - 62s 6s/step - loss: 0.5941 - acc: 0.7333 - val_loss: 0.6029 - val_acc: 0.8375

Epoch 00002: val_loss improved from 0.65439 to 0.60291, saving model to best_model.h5
Epoch 3/10
10/10 [==============================] - 62s 6s/step - loss: 0.5169 - acc: 0.7417 - val_loss: 0.5474 - val_acc: 0.8875

Epoch 00003: val_loss improved from 0.60291 to 0.54737, saving model to best_model.h5
Epoch 4/10
10/10 [==============================] - 62s 6s/step - loss: 0.4135 - acc: 0.9167 - val_loss: 0.4840 - val_acc: 0.8250

Epoch 00004: val_loss improved from 0.54737 to 0.48403, saving model to best_model.h5
Epoch 5/10
10/10 [==============================] - 62s 6s/step - loss: 0.3574 - acc: 0.9083 - val_loss: 0.4174 - val_acc: 0.8500

Epoch

In [0]:
print("--TrianSet_Evaluate--")
scores=model.evaluate_generator(train_generator,steps=24)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

--TrianSet_Evaluate--
acc: 93.33%


In [0]:
print("--TestSet_Evaluate--")

scores=model.evaluate_generator(test_generator,steps=16)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

--TestSet_Evaluate--
acc: 87.50%
